# In this tutorial the operator "mapdl::run" is used to generate an rst file and then, results are evaluated on this rst with a cyclic model

## lauch the server

In [1]:
from ansys import dpf
dpf.start_local_server()

server started at --address 10.110.2.47 --port 50052


## use "mapdl::run"

In [26]:
import os

if 'AWP_UNIT_TEST_FILES' in os.environ:
    unit_test_files = os.environ['AWP_UNIT_TEST_FILES']
    
test_file_path = os.path.join(unit_test_files, 'DataProcessing', 'cyclic','quad','ds.dat')
data_sources = dpf.DataSources(test_file_path)

In [27]:
mapld_run =dpf.Operator("mapdl::run")
#print(mapld_run)

In [28]:
mapld_run.inputs.data_sources(data_sources)
mapld_run.inputs.mapdl_exe_path(r'C:\\Program Files\\ANSYS Inc\\v212\\ansys\\bin\\winx64\\ansys2021R2.exe')
rst_data_sources = mapld_run.outputs.data_sources()

## read diplacement, stress and strain

In [29]:
model = dpf.Model(rst_data_sources)
time_freq_support =model.time_freq_support
#print(time_freq_support)

In [30]:
assert time_freq_support.n_sets==30

In [31]:
mesh =model.meshed_region
#print(mesh)

In [32]:
els =mesh.elements
el = mesh.get_element(elementid=1)

In [33]:
assert len(els.ids)==4

In [34]:
assert el.nodes_count()==20
assert el.get_nodes()[0].coordinates == [-1.084202172e-18, 0.00102, 0.002921563606]
assert el.get_nodes()[3].coordinates == [-0.001210151269, 0.00102, 0.002921563606]

In [35]:
el = mesh.get_element(elementid=2)

In [36]:
assert el.nodes_count()==20
assert el.get_nodes()[0].coordinates == [-1.084202172e-18, 0.00102, 0.002921563606]
assert el.get_nodes()[3].coordinates == [-1.084202172e-18, 0.0, 0.002921563606]

### stress nodal

In [37]:
s = model.operator("mapdl::rst::S_cyclic")
s.inputs.requested_location("Nodal")
fcS = s.outputs.fields_container()

In [38]:
min_max= dpf.Operator("min_max")
min_max.inputs.field.connect(fcS[0])

In [39]:
min = min_max.outputs.field_min()
max =  min_max.outputs.field_max()

In [40]:
assert min.data ==[-1.612489189611259e+16,
 -7282652033180194.0,
 -1.6124891896112584e+16,
 -8475938880878909.0,
 -8475938880878907.0,
 -4549709949260746.0]

assert max.data ==[1.6124891896112594e+16,
 7282652033180192.0,
 1.6124891896112588e+16,
 8475938880878907.0,
 8475938880878907.0,
 4549709949260745.0]

In [41]:
assert len(fcS[0].scoping.ids)==304

### stress elemental nodal

In [42]:
s = model.operator("mapdl::rst::S_cyclic")
fcS = s.outputs.fields_container()
min_max= dpf.Operator("min_max")
min_max.inputs.field.connect(fcS[0])
min = min_max.outputs.field_min()
max =  min_max.outputs.field_max()

In [43]:
assert min.data ==[-1.6124891896112592e+16, -7282652033180194.0, -1.6124891896112584e+16, -8475938880878909.0, -8475938880878907.0, -4549709949260747.0]

assert max.data ==[1.6124891896112596e+16, 7282652033180192.0, 1.6124891896112588e+16, 8475938880878907.0, 8475938880878907.0, 4549709949260746.0]

### displacement 

In [45]:
u = model.operator("mapdl::rst::U_cyclic")
u.inputs.time_scoping([1,2,3])
fcU = u.outputs.fields_container()

In [46]:
min_max_over_time= dpf.Operator("min_max_fc")
min_max_over_time.inputs.fields_container.connect(fcU)
min = min_max_over_time.outputs.field_min()
max =  min_max_over_time.outputs.field_max()

In [47]:
assert min.data ==[-45.61903533293341, 0.0, -45.61903533293339, -73.80905178031536, -3.500528202108204, -73.80905178031537, -100.78111048835589, -43.43288110302569, -100.78111048835592]

assert max.data ==[45.6190353329334, 94.52618694714785, 45.61903533293341, 73.80905178031536, 3.500528202122901, 73.80905178031536, 100.78111048835589, 43.43288109112891, 100.78111048835589]

## delete the temp folder

In [48]:
path =os.path.join(rst_data_sources.result_files[0])
import shutil
shutil.rmtree(os.path.dirname(path))